In [1]:
#!/home/ubuntu/anaconda3/bin/python

from binance.client import Client
import time
import dateparser
import pytz
from datetime import datetime
import json
import pandas as pd

from backtesting import Strategy
from backtesting.lib import crossover

from backtesting import Backtest
from backtesting.test import GOOG


def date_to_milliseconds(date_str):
    """Convert UTC date to milliseconds
    If using offset strings add "UTC" to date string e.g. "now UTC", "11 hours ago UTC"
    See dateparse docs for formats http://dateparser.readthedocs.io/en/latest/
    :param date_str: date in readable format, i.e. "January 01, 2018", "11 hours ago UTC", "now UTC"
    :type date_str: str
    """
    # get epoch value in UTC
    epoch = datetime.utcfromtimestamp(0).replace(tzinfo=pytz.utc)
    # parse our date string
    d = dateparser.parse(date_str)
    # if the date is not timezone aware apply UTC timezone
    if d.tzinfo is None or d.tzinfo.utcoffset(d) is None:
        d = d.replace(tzinfo=pytz.utc)

    # return the difference in time
    return int((d - epoch).total_seconds() * 1000.0)


def interval_to_milliseconds(interval):
    """Convert a Binance interval string to milliseconds
    :param interval: Binance interval string 1m, 3m, 5m, 15m, 30m, 1h, 2h, 4h, 6h, 8h, 12h, 1d, 3d, 1w
    :type interval: str
    :return:
         None if unit not one of m, h, d or w
         None if string not in correct format
         int value of interval in milliseconds
    """
    ms = None
    seconds_per_unit = {
        "m": 60,
        "h": 60 * 60,
        "d": 24 * 60 * 60,
        "w": 7 * 24 * 60 * 60
    }

    unit = interval[-1]
    if unit in seconds_per_unit:
        try:
            ms = int(interval[:-1]) * seconds_per_unit[unit] * 1000
        except ValueError:
            pass
    return ms

def get_historical_klines(symbol, interval, start_str, end_str=None):
    """Get Historical Klines from Binance
    See dateparse docs for valid start and end string formats http://dateparser.readthedocs.io/en/latest/
    If using offset strings for dates add "UTC" to date string e.g. "now UTC", "11 hours ago UTC"
    :param symbol: Name of symbol pair e.g BNBBTC
    :type symbol: str
    :param interval: Biannce Kline interval
    :type interval: str
    :param start_str: Start date string in UTC format
    :type start_str: str
    :param end_str: optional - end date string in UTC format
    :type end_str: str
    :return: list of OHLCV values
    """
    # create the Binance client, no need for api key
    client = Client("", "")

    # init our list
    output_data = []

    # setup the max limit
    limit = 500

    # convert interval to useful value in seconds
    timeframe = interval_to_milliseconds(interval)

    # convert our date strings to milliseconds
    start_ts = date_to_milliseconds(start_str)

    # if an end time was passed convert it
    end_ts = None
    if end_str:
        end_ts = date_to_milliseconds(end_str)

    idx = 0
    # it can be difficult to know when a symbol was listed on Binance so allow start time to be before list date
    symbol_existed = False
    while True:
        # fetch the klines from start_ts up to max 500 entries or the end_ts if set
        temp_data = client.get_klines(
            symbol=symbol,
            interval=interval,
            limit=limit,
            startTime=start_ts,
            endTime=end_ts
        )

        # handle the case where our start date is before the symbol pair listed on Binance
        if not symbol_existed and len(temp_data):
            symbol_existed = True

        if symbol_existed:
            # append this loops data to our output data
            output_data += temp_data

            # update our start timestamp using the last value in the array and add the interval timeframe
            start_ts = temp_data[len(temp_data) - 1][0] + timeframe
        else:
            # it wasn't listed yet, increment our start date
            start_ts += timeframe

        idx += 1
        # check if we received less than the required limit and exit the loop
        if len(temp_data) < limit:
            # exit the while loop
            break

        # sleep after every 3rd call to be kind to the API
        if idx % 3 == 0:
            time.sleep(1)

    return output_data


def get_ohlc(symbol, start, end="1 hour ago", interval=Client.KLINE_INTERVAL_1DAY):
    start_str = start + " UTC"
    end_str = end + " UTC"
    klinesa = get_historical_klines(symbol, interval,start_str, end_str)
    start_ts = date_to_milliseconds(start_str)
    cnames = ['open_time', 'Open', 'High', 'Low', 'Close', 'volume', 'close_time', 
              'volume', 'n_trades',' taker_buybase_volume', 'taker_buyquote_volume', 'ignore']
    odf = pd.DataFrame.from_dict(klinesa)
    odf.columns = cnames
    outdf = odf.copy()[['Open','High', 'Low', 'Close']]
    outdf.index = pd.to_datetime(odf['open_time'], unit='ms')
    cols = outdf.columns
    outdf[cols] = outdf[cols].apply(pd.to_numeric, errors='coerce')
    return outdf


from backtesting.test import SMA

import pandas as pd
from backtesting.lib import SignalStrategy, TrailingStrategy


class SmaCross(SignalStrategy,
               TrailingStrategy):
    n1 = 10
    n2 = 20
    atr = 1
    def init(self):
        # In init() and in next() it is important to call the
        # super method to properly initialize all the classes
        super().init()
        
        # Precompute the two moving averages
        sma1 = self.I(SMA, self.data.Close, self.n1)
        sma2 = self.I(SMA, self.data.Close, self.n2)
        
        # Taking a first difference (`.diff()`) of a boolean
        # series results in +1, 0, and -1 values. In our signal,
        # as expected by SignalStrategy, +1 means buy,
        # -1 means sell, and 0 means to hold whatever current
        # position and wait. See the docs.
        signal = (pd.Series(sma1) > sma2).astype(int).diff().fillna(0)
        
        # Set the signal vector using the method provided
        # by SignalStrategy
        self.set_signal(signal)
        
        # Set trailing stop-loss to 4x ATR
        # using the method provided by TrailingStrategy
        self.set_trailing_sl(self.atr)
        
def get_opt_params(stats):
    oreturn = str(stats['Return [%]'])
    on1=str(stats['_strategy']).split("=")[1].split(",")[0]
    on2=str(stats['_strategy']).split("=")[2].split(",")[0]
    oatr=str(stats['_strategy']).split("=")[3].split(",")[0].split(")")[0]
    return oreturn, on1, on2, oatr


def backtest_windows(nmonths, symbol, interv, period):
    for month in range(nmonths):
        end_month = month - period
        if end_month > 0:
            df = get_ohlc(symbol, start=str(month) + " months ago", end = str(end_month) + " months ago", interval=interv)
            bt = Backtest(df, SmaCross, commission=0.09)
            stats = bt.optimize(n1=range(100, 500, 10),
                        n2=range(400, 1000, 10),
                        atr=range(1, 30, 2),
                        maximize='Equity Final [$]',
                        constraint=lambda p: p.n1 < p.n2)
            oreturn, on1, on2, oatr = get_opt_params(stats)
            outdf = pd.DataFrame({'return': oreturn, 'n1': on1, 'n2': on2, 'atr': oatr, 
                                  'month': str(month), 'interval': str(interv).split("_")[-1], 
                                  'symbol': symbol, 'period': str(period)})
            outdf.to_csv("/home/ubuntu/binance_data/optimized_april26.txt", mode='a', header=False, index=False, sep="\t")
    return True



# symbols = ['ETHBTC', 'BTCUSDT', 'ETHUSDT']
# interv = Client.KLINE_INTERVAL_30MINUTE
# period = 4
# nmonths = 24
# for symbol in symbols:
#     print("running for: " + symbol)
#     backtest_windows(nmonths, symbol, interv, period)
    


/opt/anaconda3/lib/python3.7/site-packages/backtesting/_plotting.py:44: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [ ]:


symbols = ['ETHBTC', 'BTCUSDT', 'ETHUSDT']
interv = Client.KLINE_INTERVAL_30MINUTE
period = 4
nmonths = 24
for symbol in symbols:
    print("running for: " + symbol)
    backtest_windows(nmonths, symbol, interv, period)
    

In [8]:
def get_opt_params(stats):
    oreturn = str(stats['Return [%]'])
    on1=str(stats['_strategy']).split("=")[1].split(",")[0]
    on2=str(stats['_strategy']).split("=")[2].split(",")[0]
    oatr=str(stats['_strategy']).split("=")[3].split(",")[0].split(")")[0]
    return oreturn, on1, on2, oatr


def backtest_windows(nmonths, symbol, interv, period):
    for month in range(nmonths):
        end_month = month - period
        if end_month > 0:
            df = get_ohlc(symbol, start=str(month) + " months ago", end = str(end_month) + " months ago", interval=interv)
            bt = Backtest(df, SmaCross, commission=0.09)
            stats = bt.optimize(n1=range(100, 500, 25),
                        n2=range(400, 1000, 25),
                        atr=range(1, 30, 2),
                        maximize='Equity Final [$]',
                        constraint=lambda p: p.n1 < p.n2)
            oreturn, on1, on2, oatr = get_opt_params(stats)
            outdf = pd.DataFrame({'return': oreturn, 'n1': on1, 'n2': on2, 'atr': oatr, 
                                  'month': str(month), 'interval': str(interv).split("_")[-1], 
                                  'symbol': symbol, 'period': str(period)})
            outdf.to_csv("~/coding/aws_trade/binance_data/optimized_april26.txt", mode='a', header=False, index=False, sep="\t")
    return True


In [9]:
btcusdt = get_ohlc("ETHBTC", "1 month ago", interval=Client.KLINE_INTERVAL_15MINUTE)
print(len(btcusdt))

2962


In [ ]:
backtest_windows(3, "ETHBTC", Client.KLINE_INTERVAL_1HOUR, 1)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: Searching best of 5610 configurations.
